In [2]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float)
    .scatter_(0, torch.tensor(y), value=1)) 
    # .scatter_(dim, index, src) 0: row, 1: column
)

# 신경망 모델

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = (
    "cuda"
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)
print(f"Using {device} device")

Using mps device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='mps:0')


In [8]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [13]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.6582, -0.1353, -0.0874,  0.0507,  0.7552,  0.3282,  0.1547,  0.2286,
          0.0469, -0.4693,  0.0982, -0.0197, -0.2532,  0.7054,  0.4125, -0.4398,
         -0.0740,  0.0463,  0.2448,  0.0824],
        [-0.5879, -0.0773,  0.4313,  0.1517,  0.4963,  0.5738,  0.1466,  0.1270,
          0.3046, -0.4177, -0.0917, -0.1849,  0.0173,  0.2320, -0.2136, -0.2394,
         -0.1268,  0.2571,  0.5050, -0.0157],
        [-0.6701,  0.0487,  0.2565,  0.2354,  0.3989,  0.5591, -0.2334,  0.3059,
          0.2228, -0.0570, -0.2449, -0.2151, -0.3501,  0.3250,  0.3324, -0.3523,
          0.0409, -0.0267,  0.4267, -0.1337]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0507, 0.7552, 0.3282, 0.1547, 0.2286, 0.0469,
         0.0000, 0.0982, 0.0000, 0.0000, 0.7054, 0.4125, 0.0000, 0.0000, 0.0463,
         0.2448, 0.0824],
        [0.0000, 0.0000, 0.4313, 0.1517, 0.4963, 0.5738, 0.1466, 0.1270, 0.3046,
         0.0000, 0.0000, 0.0000, 0.0173, 0.2320, 0.00

In [16]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20 ,10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [19]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [21]:
print(f"Model structure: {model}\n\n")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0277, -0.0314,  0.0257,  ..., -0.0149,  0.0343, -0.0282],
        [ 0.0232,  0.0301,  0.0281,  ..., -0.0179, -0.0255, -0.0292]],
       device='mps:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([0.0008, 0.0060], device='mps:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0312,  0.0220, -0.0137,  ...,  0.0058, -0.0119,  0.0042],
        [ 0.0086,  0.0058,  0.0396,  ..., -0.0316,  0.0062, -0.0216]],
       device='mps:0', grad_fn=<SliceBackwar

# 자동 미분